# Metropolis. Un proyecto software

V1.0

## Objetivo general

Simular la **evolución de un proyecto de software** llamado `Metropolis`, que implementa el **algoritmo de Metropolis** (núcleo del Simulated Annealing), en tres versiones:

* **v1.0:** implementación básica del algoritmo.
* **v2.0:** refinamiento modular, incorporación de criterios adaptativos de temperatura y registro de resultados.
* **v3.0:** optimización, documentación y graficación de convergencia.

Todo con control de versiones en Git y sincronización con un **repositorio remoto en GitHub** bajo la cuenta `Chema-ES`.

## Enfoque 

En el esquema propuesto se usarán tags (etiquetas), no branches, para marcar los “snapshots” o hitos de versión (v1.0, v2.0, v3.0).

Esto reproduce de forma realista el ciclo de vida secuencial de releases estables, donde cada versión publicada del software se marca con un tag, mientras el desarrollo continúa sobre la rama principal (main).

Los **tags** marcarán versiones *publicadas o estables*

Son **inmutables** (fotografías exactas del código en cierto punto).

```bash
git tag v1.0   # primera versión estable
git tag v2.0   # versión refinada
git tag v3.0   # versión final
```

Estas etiquetas no se usan para desarrollo, sino para **referenciar entregas concretas**, compilar releases, o volver atrás fácilmente.

La opción de branches no es necesaria en este caso, ya que no se consideran desarrollos paralelos al main, como podrían ser los módulos de visualización o scheduling.

Los **branches (ramas)** en Git se utilizan principalmente cuando:

1. Hay **desarrollos paralelos**

Cada rama representa una línea de trabajo independiente que puede evolucionar sin interferir con el resto del código base.

**Ejemplo típico:**

```bash
main                   # versión estable del software
feature-schedule       # nuevo tipo de enfriamiento
feature-visualization  # módulo de graficación
hotfix-energy          # corrección urgente en producción
```

Cada una avanza por su cuenta, y luego se integran (merge) a `main` cuando están listas.

2. Hay **varios colaboradores**

Cada desarrollador puede tener su propia rama, evitando conflictos:

```bash
Chema-ES/feature-logging
AnaDev/feature-ui
LuisQA/tests-integration
```

Esto permite trabajar de forma **asíncrona y segura**, sin romper la versión estable en `main`.

Cuando el trabajo está revisado y aprobado, se hace un:

```bash
git merge feature-logging
```

o en GitHub, un **Pull Request (PR)**.




## Estructura repositorio

```text
Metropolis/
├── README.md
├── LICENSE
├── requirements.txt
├── metropolis/
│   ├── __init__.py
│   ├── core.py          # núcleo del algoritmo
│   ├── schedule.py      # funciones de temperatura (v2+)
│   └── visualization.py # para plots de convergencia (v3)
├── examples/
│   └── example_run.py
└── tests/
    └

## Fase 1 – V1.0: Implementación básica del algoritmo Metropolis

### Código principal: core.py

```python
import math
import random

def metropolis_step(current_state, current_energy, neighbor_fn, energy_fn, T):
    """Ejecuta un paso del algoritmo de Metropolis."""
    new_state = neighbor_fn(current_state)
    new_energy = energy_fn(new_state)
    delta_E = new_energy - current_energy

    if delta_E < 0 or random.random() < math.exp(-delta_E / T):
        return new_state, new_energy
    return current_state, current_energy

def simulated_annealing(initial_state, energy_fn, neighbor_fn, T0, cooling_rate, steps):
    state = initial_state
    energy = energy_fn(state)
    T = T0

    for _ in range(steps):
        state, energy = metropolis_step(state, energy, neighbor_fn, energy_fn, T)
        T *= cooling_rate

    return state, energy
```

### Ejemplo: example_run.py

```python
from metropolis.core import simulated_annealing
import random

def energy_fn(x): return x**2
def neighbor_fn(x): return x + random.uniform(-1, 1)

best_state, best_energy = simulated_annealing(
    initial_state=5.0, energy_fn=energy_fn,
    neighbor_fn=neighbor_fn, T0=10, cooling_rate=0.99, steps=1000
)

print("Best state:", best_state, "Energy:", best_energy)
```

## Fase 2 – V2.0: Refinamiento + schedule adaptable + logs

### Novedades

* Nuevo módulo `schedule.py` con distintos esquemas de enfriamiento.
* Logging de energía para análisis posterior.
* Separación de configuración y ejecución.

### Código nuevo: schedule.py

```python
# metropolis/schedule.py
import math

def exponential(T0, rate, step):
    return T0 * (rate ** step)

def linear(T0, rate, step):
    return max(T0 - rate * step, 1e-6)

def logarithmic(T0, step):
    return T0 / math.log(2 + step)
```

### Código actualizado: core.py
```python
# metropolis/core.py (actualizado)
import random, math
from metropolis import schedule

def simulated_annealing(initial_state, energy_fn, neighbor_fn, schedule_fn, T0, steps):
    state = initial_state
    energy = energy_fn(state)
    history = []

    for step in range(steps):
        T = schedule_fn(T0, step)
        new_state = neighbor_fn(state)
        new_energy = energy_fn(new_state)
        delta_E = new_energy - energy

        if delta_E < 0 or random.random() < math.exp(-delta_E / T):
            state, energy = new_state, new_energy

        history.append((step, T, energy))
    return state, energy, history
```


## Fase 3 – Versión 3.0: Optim + vis + doc

### Novedades

* Módulo `visualization.py` para graficar convergencia.
* `README.md` completo y docstrings.
* Tests unitarios en `tests/test_core.py`.

### Código:plot_history
```python
# metropolis/visualization.py
import matplotlib.pyplot as plt

def plot_history(history):
    steps, T, E = zip(*[(s, t, e) for s, t, e in history])
    fig, ax1 = plt.subplots()
    ax1.plot(steps, E, label="Energy", color="tab:blue")
    ax2 = ax1.twinx()
    ax2.plot(steps, T, label="Temperature", color="tab:orange", alpha=0.5)
    ax1.set_xlabel("Step")
    ax1.set_ylabel("Energy")
    ax2.set_ylabel("Temperature")
    plt.title("Simulated Annealing Convergence")
    plt.show()
```


## Extras opcionales

* Añadir CI/CD con GitHub Actions para test automático.
* Publicar documentación con MkDocs o Sphinx.
* Añadir ejemplo Jupyter Notebook (`notebooks/Metropolis_demo.ipynb`).

---

¿Quieres que te genere **el repositorio completo en formato comprimido (.zip)** con las tres versiones preparadas (cada una en su rama o tag), para que solo tengas que subirlo a GitHub con `gh repo create`?
Puedo hacerlo y dejarte los comandos exactos para desplegarlo.

# Anexo. comandos git

## Comandos git iniciales

```bash

# Crear repo local
mkdir Metropolis && cd Metropolis
git init
```
El sistema responderáa con una serie de mensajes:

```
ayuda: Usando 'master' como el nombre de la rama inicial. Este nombre de rama predeterminado
ayuda: está sujeto a cambios. Para configurar el nombre de la rama inicial para usar en todos
ayuda: de sus nuevos repositorios, reprimiendo esta advertencia, llama a:
ayuda: 
ayuda: 	git config --global init.defaultBranch <nombre>
ayuda: 
ayuda: Los nombres comúnmente elegidos en lugar de 'master' son 'main', 'trunk' y
ayuda: 'development'. Se puede cambiar el nombre de la rama recién creada mediante este comando:
ayuda: 
ayuda: 	git branch -m <nombre>
```
## Release V1.0
**(después de escribir `core.py` y `example_run.py`)**
**Añadir v1.0**
git add .
git commit -m "v1.0: Implementación básica del algoritmo Metropolis"

**Crear repo remoto en GitHub**
gh repo create Chema-ES/Metropolis --public --source=. --remote=origin

git push -u origin main
git tag v1.0
git push origin v1.0

## Release V2.0
**(tras modificar e introducir schedule.py y logs)**
```bash
git add .
git commit -m "v2.0: Modularización y funciones de enfriamiento adaptativo"
git push
git tag v2.0
git push origin v2.0
```
## Release V3.0
**(tras añadir visualización, documentación y tests)**
```bash
git add .
git commit -m "v3.0: Visualización y optimización final"
git push
git tag v3.0
git push origin v3.0
```